# Introducción a modelos de regresión

La regresión logística **(RL)** forma parte del conjunto de métodos estadísticos que caen bajo tal
denominación y es la variante que corresponde al caso en que se valora la contribución de diferentes
factores en la ocurrencia de un evento simple.


En general, la regresión logística es adecuada cuando la **variable de respuesta Y es politómica**
(admite varias categorías de respuesta, tales como mejora mucho, empeora, se mantiene, mejora,
mejora mucho), pero es especialmente útil en particular cuando solo hay dos **posibles respuestas**
(cuando la variable de respuesta es dicotómica), que es el caso más común.


La **RL** es una de las técnicas estadístico‐inferenciales más empleadas en la producción científica
contemporánea. Surge en la década de los **60's**, su generalización dependía de la solución que se diera al
problema de la **estimación de los coeficientes.**

El algoritmo de **Walker‐Duncan** para la obtención de
los estimadores de máxima verosimilitud vino a solucionar en parte este problema, pero era de
naturaleza tal que el uso de computadoras era imprescindible.


La RL va  a contestar a preguntas tales como: 

   1. ¿Se puede predecir con antelación si un cliente que
solicita un préstamo a un banco va a ser un cliente moroso?.

   2. ¿Se puede predecir si una empresa va a
entrar en bancarrota?.

   3. **¿Se puede predecir de antemano que un paciente corra riesgo de un infarto?.**

La identificación del mejor modelo de regresión logística se realiza mediante la comparación de
modelos utilizando el cociente de **verosimilitud**, que indica a partir de los datos de la muestra **cuanto
más probable es un modelo frente al otro.**

La diferencia de los cocientes de verosimilitud entre dos
modelos se distribuye según la ley de la Chi‐cuadrado con los grados de libertad correspondientes a
**la diferencia en el número de variables entre ambos modelos.**

Si a partir de este coeficiente no se puede demostrar que un modelo resulta mejor que el otro, se
considerará como el más adecuado, el más sencillo.

# **Objetivo:**

En este caso analizaremos un dataset de Kaggle en el cual viene la informacion de **4237** personas con las siguientes caracteristicas:

  1.  genero.

  2.  edad.

  3.  educacion.

  4. FumadorActivo.

  5. Cigarros por dia.

  6.  Medidas BPM(Beats por Minuto)

  7.  Humo prevalente.

  8.  Hyp prevalente.

  9.  Diabetes.

  10. totChol.

  11. sysBP

  12.  diaBP	

  13. BMI

  14. Ritmo Cardiaco.

  15. Glucosa.

  16. Diez años CHD


Nuestro objetivo es crear un **modelo para predecir si una persona corre el riesgo o no de sufrir un infarto**, teniendo en cuanta que dependemos de 2 variables , en este caso seran 0 & 1.

## Primera parte: Analizar los datos

### Importamos las librerías necesarias

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error,r2_score 
import matplotlib.pyplot as plt

### Importamos nuestro conjunto de datos

In [2]:
enfermedad_corazon = pd.read_csv('/Users/familiavs/Downloads/heart_disease.csv')

### Analizamos los datos y metadatos de nuestro data frame

In [11]:
enfermedad_corazon.tail()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0


In [12]:
enfermedad_corazon.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [13]:
enfermedad_corazon=enfermedad_corazon.dropna()
enfermedad_corazon.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [14]:
enfermedad_corazon.corr()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
male,1.000000,-0.024387,0.017677,0.206778,0.331243,-0.052128,-0.002308,0.000806,0.013833,-0.070229,-0.045484,0.051575,0.072867,-0.114923,0.003048,0.091745
age,-0.024387,1.000000,-0.158961,-0.210862,-0.189099,0.134670,0.050864,0.306693,0.109027,0.267764,0.388551,0.208880,0.137172,-0.002685,0.118245,0.233810
education,0.017677,-0.158961,1.000000,0.025253,0.013527,-0.013647,-0.030353,-0.079100,-0.039547,-0.012956,-0.124511,-0.058502,-0.137280,-0.064254,-0.031874,-0.063068
currentSmoker,0.206778,-0.210862,0.025253,1.000000,0.773819,-0.051936,-0.038159,-0.107561,-0.041859,-0.051119,-0.134371,-0.115748,-0.159574,0.050452,-0.053346,0.019176
cigsPerDay,0.331243,-0.189099,0.013527,0.773819,1.000000,-0.046479,-0.036283,-0.069890,-0.036934,-0.030222,-0.094764,-0.056650,-0.086888,0.063549,-0.053803,0.052159
BPMeds,-0.052128,0.134670,-0.013647,-0.051936,-0.046479,1.000000,0.113119,0.263047,0.049051,0.094011,0.271291,0.199750,0.105603,0.012894,0.054210,0.089116
prevalentStroke,-0.002308,0.050864,-0.030353,-0.038159,-0.036283,0.113119,1.000000,0.066098,0.009619,0.012697,0.061080,0.055878,0.036478,-0.017020,0.016051,0.048351
prevalentHyp,0.000806,0.306693,-0.079100,-0.107561,-0.069890,0.263047,0.066098,1.000000,0.080623,0.167074,0.697790,0.617634,0.302917,0.147333,0.087129,0.181556
diabetes,0.013833,0.109027,-0.039547,-0.041859,-0.036934,0.049051,0.009619,0.080623,1.000000,0.048371,0.102574,0.050767,0.088970,0.060996,0.614817,0.093397
totChol,-0.070229,0.267764,-0.012956,-0.051119,-0.030222,0.094011,0.012697,0.167074,0.048371,1.000000,0.220130,0.174986,0.120799,0.093057,0.049749,0.091127


In [15]:
print(enfermedad_corazon.groupby('TenYearCHD').size())

TenYearCHD
0    3099
1     557
dtype: int64


## Segunda parte: Modelo de regresión logística

In [16]:
X=enfermedad_corazon[enfermedad_corazon.columns]
X=X.drop(columns=['TenYearCHD'])
X.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0


In [17]:
Y=enfermedad_corazon['TenYearCHD']
Y.head()

0    0
1    0
2    0
3    1
4    0
Name: TenYearCHD, dtype: int64

In [18]:
X=enfermedad_corazon[['age','cigsPerDay','totChol','sysBP','diaBP','glucose']]
X.head()

,age,cigsPerDay,totChol,sysBP,diaBP,glucose
0,39,0.0,195.0,106.0,70.0,77.0
1,46,0.0,250.0,121.0,81.0,76.0
2,48,20.0,245.0,127.5,80.0,70.0
3,61,30.0,225.0,150.0,95.0,103.0
4,46,23.0,285.0,130.0,84.0,85.0


### Separamos nuestros datos en entrenamiento y pruebas

In [19]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

### Creamos nuestro modelo y lo entrenamos

In [24]:
#En esta seccion aplicamos el metodo LogisticRgeression(max_iter=1) para que converga y genere un mejor modelo, en este caso , como le pusimos 1 como parametro solo entreno 1 vez , pero si lo cambiamos que pasara?

# mi_modelo=LogisticRegression(max_iter=1)

mi_modelo=LogisticRegression(max_iter=1000)

#Podemos ver que si subimos el parametro de iteracion nuestro modelo mejora, pero es muy tedioso cambiar este parametro, para eso vamos a automatizarlo, es importante leer la documentacion de sklearn para entender mejor los metodos.

#Usamos el metodo fit para entrenar nuesto modelo con los datos de entrenamiento.
mi_modelo.fit(x_train,y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Verificamos el accuracy de nuestro modelo

In [25]:
result=mi_modelo.score(x_test,y_test)
print('Accuracy : ' ,(result))

Accuracy :  0.8579234972677595


# AJUSTE DE HIPERPARAMETROS.

Los **hiperparámetros** de un modelo son los **valores** de las configuraciones utilizadas durante el proceso de entrenamiento. Son valores que generalmente se **no se obtienen de los datos**, por lo que suelen ser indicados por el científico de datos. 

El valor óptimo de un hiperparámetro no se puede conocer a priori para un problema dado. Por lo que se tiene que utilizar valores genéricos, reglas genéricas, los valores que han funcionado anteriormente en problemas similares o buscar la mejor opción mediante prueba y error. Siendo una buena opción buscar los hiperparámetros la validación cruzada.



In [26]:
from sklearn.model_selection import GridSearchCV

In [30]:
parametros = {"C":[1,10],"penalty":['11','12'],"max_iter":[10,20,30,40,50,60,70,80,90,100]}

In [31]:
logreg_cv = GridSearchCV(mi_modelo,parametros,cv=10)